In [1]:
%gui asyncio
import asyncio
import functools
import threading
import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import time
import re
import serial
import sys
import glob
import serial
import numpy as np

In [2]:
## CONSTANTS ##
CMDPROMPT='>>: '
LINETERM='\r\n'
currentposition=[0,0,0]
myport = None

In [3]:
#https://stackoverflow.com/questions/12090503/listing-available-com-ports-with-python
def get_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

In [4]:
def list_ports():
    print('Available serial ports: ')
    ports=get_ports()
    for port in ports:
        print(ports.index(port), end=':')
        print(port)
def try_connect(portindex):
    myport = None
    try:
        myport = serial.Serial(
            port=get_ports()[portindex],
            baudrate=250000,
            timeout=0,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            bytesize=serial.EIGHTBITS
        )
        print('Sucessfully connected to port '+myport.port)
        return myport
    except (OSError, serial.SerialException):
        print('ERROR: ', end='')
        if myport.is_open:
            print('Port already open')
        elif myport.port not in portlist:
            print('Port '+myport.port+' not found')
        else:
            print('Could not connect to port')
        pass

In [5]:
myport=try_connect(1)

Sucessfully connected to port COM5


In [6]:
def read_serial():
    return myport.readline().decode().rstrip()

def contains_ok(lines):
    if 'ok' in lines:
        return True
    return False

def handle_input(sender):
    userinput=sender.value
    sender.value=''
    if userinput in macros:
        terminalout.append_display_data("MACRO: "+userinput)
        macros[userinput]()
    else:
        send_gcode(userinput, False)
def send_gcode(gcode, wait=False):
    if wait:
        userin.disabled=True
        userin.placeholder='Waiting for OK'
    raw = gcode.upper()
    machineout.append_display_data('SEND: '+raw)
    sInput = (raw + LINETERM) #append terminator string
    bInput = sInput.encode() #convert string to bytes
    myport.write(bInput) #send the command

def current_milli_time():
    return round(time.time() * 1000)

def is_m114_feedback(printeroutput):
    if(len(re.findall("(?:[^E][:])(\d{1,3}[.]\d{4})", printeroutput))==3):
        return True
    return False

def parse_coords(printeroutput):
    return list(map(float, re.findall("(?:[^E][:])(\d{1,3}[.]\d{4})",printeroutput)))

In [7]:
## MACRO DEFINITIONS ##
def show_help(b=None):
    """Shows this help"""
    terminalout.append_display_data('Available macros:')
    for macro in macros.items():
        terminalout.append_display_data(macro[0]+': '+macro[1].__doc__)

def probe_grid(b=None):
    """Returns array of probed coordinate"""
    
def probe_point(b=None):
    """Returns the probed coordinate"""

def home(b=None):
    """Auto Home XYZ"""
    send_gcode('G28')

def deploy_probe(b=None):
    """deploy_probe the probe"""
    send_gcode('M401')

def stow_probe(b=None):
    """stow_probe the probe"""
    send_gcode('M402')
    
def enable_fan(b=None):
    """Set case fan speed to MAX"""
    send_gcode('M106 P2 S255')
    
def disable_fan(b=None):
    """Disable all fans"""
    send_gcode('M106 P2 S0')
    
def get_pos(b=None):
    """Report the current tool position"""
    send_gcode('M114')
    
def disable_motors(b=None):
    """Disables all the motors"""
    send_gcode('M18')

def set_relative(b=None):
    """Set all axes to relative"""
    send_gcode('G91')

def set_absolute(b=None):
    """Set all axes to asbolute"""
    send_gcode('G90')
        
def do_jog(b=None, _pos=[0,0,0]):
    """Jogs machine"""
    set_relative()
    send_gcode('G00 X'+str(_pos[0])+' Y'+str(_pos[1])+' Z'+str(_pos[2]), False)
    set_absolute()
    
## MACRO DICTIONARY
macros = {
    'help':show_help,
    'probe':probe_point,
    'grid':probe_grid,
    'home':home,
    'deploy_probe':deploy_probe,
    'stow_probe':stow_probe,
    'fon':enable_fan,
    'foff':disable_fan,
    'pos':get_pos,
    'moff':disable_motors,
    'rel':set_relative,
    'abs':set_absolute
}

In [8]:
## INPUT HOOK FUNCTIONS ##

## INPUT HOOK DICTIONARY##


In [9]:
machineout = Output(
    layout={
            'width':'50%',
            'height':'500px',
            'border': '1px solid white',
            'overflow':'scroll',
           }
)
terminalout = Output(
    layout={
            'width':'100%',
            'height':'75%',
            'border': '1px solid white',
            'overflow':'scroll',
           }
)
userin = Text(
    layout={
            'width':'100%',
            'border': '1px solid white',
            'description':CMDPROMPT,
            'placeholder':'Type a command here'
           },
    value='',
    placeholder='Type a command here',
    disabled=False
)
buttongrid = GridspecLayout(3, 6, layout=Layout(width='100%', height='25%'))

buttongrid[0, 0] = fanbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Disable Fan',
    icon='fan',
    layout=Layout(width='90%', height='90%')
)
buttongrid[0, 2] = motorbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Disable Motors',
    icon='power-off',
    layout=Layout(width='90%', height='90%')
)
buttongrid[1, 0] = leftbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Jog X-',
    icon='arrow-left',
    layout=Layout(width='90%', height='90%')
)
buttongrid[1, 2] = rightbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Jog X+',
    icon='arrow-right',
    layout=Layout(width='90%', height='90%')
)
buttongrid[0, 1] = upbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Jog Y+',
    icon='arrow-up',
    layout=Layout(width='90%', height='90%')
)
buttongrid[2, 1] = downbutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Jog Y-',
    icon='arrow-down',
    layout=Layout(width='90%', height='90%')
)
buttongrid[1, 1] = homebutton = Button(
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Home',
    icon='home',
    layout=Layout(width='90%', height='90%')
)

userin.on_submit(handle_input)
leftbutton.on_click(functools.partial(do_jog, _pos=[-10,0,0]))
rightbutton.on_click(functools.partial(do_jog, _pos=[10,0,0]))
upbutton.on_click(functools.partial(do_jog, _pos=[0,10,0]))
downbutton.on_click(functools.partial(do_jog, _pos=[0,-10,0]))
homebutton.on_click(home)
fanbutton.on_click(disable_fan)
motorbutton.on_click(disable_motors)

rightside = VBox([terminalout, buttongrid], layout=Layout(width='50%'))
termgroup = HBox([machineout, rightside], layout=Layout(width='100%'))
controlgroup = VBox([termgroup, userin])

In [10]:
def _update():
    while True:
        serialdata = read_serial()
        if serialdata != '':
            machineout.append_display_data('RECV: '+serialdata)
            if is_m114_feedback(serialdata):
                currentposition = parse_coords(serialdata)
                terminalout.append_display_data('POSITION: '+currentposition[0])
            elif contains_ok(serialdata):
                userin.placeholder='Type a command here'
                userin.disabled=False
        time.sleep(0.001)

In [11]:
## GLOBAL ##

myport.reset_input_buffer()
myport.reset_output_buffer()
machineout.clear_output()
terminalout.clear_output()

t_update = threading.Thread(target=_update, args=())
t_update.start()

display(controlgroup)

print('Enter your command in the box above. Press <ENTER> to send.')
print('Type "help" to see available shorcuts')


Enter your command in the box above. Press <ENTER> to send.
Type "help" to see available shorcuts
